In [1]:
import pandas as pd
import numpy as np
import util

proficiency_path = './data/tx/proficiency/'

# import column maps
map_files = [
    '2019_CAMPSTAAR1_map.csv'
    , '2019_CAMPSTAAR2_map.csv'
    , '2019_CAMPSTAAR4_map.csv'
    , '2019_CAMPSTAAR5_map.csv'
    , '2018_CAMPSTAAR1_map.csv'
    , '2018_CAMPSTAAR2_map.csv'
    , '2018_CAMPSTAAR4_map.csv'
    , '2018_CAMPSTAAR5_map.csv'
    , '2017_CAMPSTAAR1_map.csv'
    , '2017_CAMPSTAAR2_map.csv'
    , '2017_CAMPSTAAR4_map.csv'
    , '2017_CAMPSTAAR5_map.csv'
    , '2016_CAMPSTAAR1_map.csv'
    , '2016_CAMPSTAAR4_map.csv'
    , '2015_CAMPSTAAR1_map.csv'
    , '2015_CAMPSTAAR4_map.csv'
    , '2014_CAMPSTAAR1_map.csv'
    , '2014_CAMPSTAAR4_map.csv'
    , '2013_CAMPSTAAR1_map.csv'
    , '2013_CAMPSTAAR4_map.csv'
]

years = [x[:4] for x in map_files]
filenames = [x[5:15] for x in map_files]

df_map = pd.read_csv(proficiency_path + map_files[0])
df_map['year'] = df_map.apply(lambda x: years[0], axis = 1)
df_map['file'] = df_map.apply(lambda x: filenames[0], axis = 1)

for i in range(1, len(map_files)):
    temp = pd.read_csv(proficiency_path + map_files[i])
    temp['year'] = temp.apply(lambda x: years[i], axis = 1)
    temp['file'] = temp.apply(lambda x: filenames[i], axis = 1)
    df_map = df_map.append(temp, ignore_index = True, sort = True)

In [2]:
# import proficiency files
proficiency_files = [
    '2019_CAMPSTAAR1.txt'
    , '2019_CAMPSTAAR2.txt'
    , '2019_CAMPSTAAR4.txt'
    , '2019_CAMPSTAAR5.txt'
    , '2018_CAMPSTAAR1.txt'
    , '2018_CAMPSTAAR2.txt'
    , '2018_CAMPSTAAR4.txt'
    , '2018_CAMPSTAAR5.txt'
    , '2017_CAMPSTAAR1.txt'
    , '2017_CAMPSTAAR2.txt'
    , '2017_CAMPSTAAR4.txt'
    , '2017_CAMPSTAAR5.txt'
    , '2016_CAMPSTAAR1.txt'
    , '2016_CAMPSTAAR4.txt'
    , '2015_CAMPSTAAR1.txt'
    , '2015_CAMPSTAAR4.txt'
    , '2014_CAMPSTAAR1.txt'
    , '2014_CAMPSTAAR4.txt'
    , '2013_CAMPSTAAR1.txt'
    , '2013_CAMPSTAAR4.txt'
]

years = [x[:4] for x in proficiency_files]
filenames = [x[5:15] for x in proficiency_files]

df = pd.read_csv(proficiency_path + proficiency_files[0])
value_vars = list(df.columns)
value_vars.remove('CAMPUS')
df = pd.melt(df, id_vars = ['CAMPUS'], value_vars = value_vars, var_name='NAME', value_name='VALUE')
df = df[df['VALUE'] != '.']
df['year'] = df.apply(lambda x: years[0], axis = 1)
df['file'] = df.apply(lambda x: filenames[0], axis = 1)

for i in range(1, len(proficiency_files)):
    temp = pd.read_csv(proficiency_path + proficiency_files[i])
    value_vars = list(temp.columns)
    value_vars.remove('CAMPUS')
    temp = pd.melt(temp, id_vars = ['CAMPUS'], value_vars = value_vars, var_name='NAME', value_name='VALUE')
    temp = temp[temp['VALUE'] != '.']
    temp['year'] = temp.apply(lambda x: years[i], axis = 1)
    temp['file'] = temp.apply(lambda x: filenames[i], axis = 1)
    df = df.append(temp, ignore_index = True, sort = True)

In [3]:
# import school info files
school_files = [
    '2019_CREF.txt'
    , '2018_CREF.txt'
    , '2017_CREF.txt'
    , '2016_CREF.txt'
    , '2015_CREF.txt'
    , '2014_CREF.txt'
    , '2013_CREF.txt'
]

years = [x[:4] for x in school_files]

df_school = pd.read_csv(proficiency_path + school_files[0])
df_school['year'] = df_school.apply(lambda x: years[0], axis = 1)

for i in range(1, len(school_files)):
    temp = pd.read_csv(proficiency_path + school_files[i])
    temp['year'] = temp.apply(lambda x: years[i], axis = 1)
    df_school = df_school.append(temp, ignore_index = True, sort = True)

In [4]:
# join column map to proficiency data, deliberately dropping rows we excluded from the column maps
print(df.shape)
df = df.merge(df_map, on = ['year', 'file', 'NAME'])
print(df.shape)

(119340601, 5)
(71549592, 8)


In [5]:
# drop First Administration records
print(df.shape)
df = df[~df.LABEL.str.contains("First Administration")]
print(df.shape)

(71549592, 8)
(70877718, 8)


In [6]:
# drop rate rows
print(df.shape)
df = df[~df.LABEL.str.contains('Rate')]
print(df.shape)

(70877718, 8)
(42168458, 8)


In [7]:
# remove previous year's records, making exception for 2017 data coming from 2018 file
print(df.shape)
def filterYears(row):
    if row['year'] != '2017' and row['LABEL'].find(str(int(row['year']) - 1)) > 0:
        return 1
    if row['year'] == '2017' and row['LABEL'].find(str(int(row['year']) + 1)) > 0:
        return 1
    return 0
df['deleteFlag'] = df.apply(lambda x: filterYears(x), axis = 1)
df = df[df['deleteFlag'] == 0]
del df['deleteFlag']
print(df.shape)

(42168458, 8)
(23836924, 8)


In [8]:
# parse grade level
def parseGrade(row):
    eocIndex = row['LABEL'].find('EOC')
    if row['LABEL'].upper().find('GRADE 3') > 0:
        return 'Grade 3'
    if row['LABEL'].upper().find('GRADE 4') > 0:
        return 'Grade 4'
    if row['LABEL'].upper().find('GRADE 5') > 0:
        return 'Grade 5'
    if row['LABEL'].upper().find('GRADE 6') > 0:
        return 'Grade 6'
    if row['LABEL'].upper().find('GRADE 7') > 0:
        return 'Grade 7'
    if row['LABEL'].upper().find('GRADE 8') > 0:
        return 'Grade 8'
    if eocIndex > 0:
        return row['LABEL'][eocIndex:]
    return 'All Grades'

df['grade'] = df.apply(lambda x: parseGrade(x), axis = 1)
df['grade'] = df['grade'].str.replace(',', '')
df['grade'] = df['grade'].str.replace('Denominator', '')
df['grade'] = df['grade'].str.replace('Numerator', '')
df['grade'] = df['grade'].str.strip()
print(df['grade'].drop_duplicates())

0                       Grade 3
264680                  Grade 4
528368                  Grade 5
7136794                 Grade 6
7281150                 Grade 7
7407810                 Grade 8
11140857          EOC English I
11259333         EOC English II
11369581          EOC Algebra I
11551769         EOC US History
11657225            EOC Biology
62677896      EOC Reading/ELA I
62719932     EOC Reading/ELA II
69164460          EOC Reading I
69204766         EOC Reading II
69240470        EOC Reading III
69313764           EOC Geometry
69353036         EOC Algebra II
69376202          EOC Writing I
69416594         EOC Writing II
69452090        EOC Writing III
69457072    EOC World Geography
69495446      EOC World History
69581752          EOC Chemistry
69613826            EOC Physics
Name: grade, dtype: object


In [9]:
# parse group
def parseGroup(row):
    if row['LABEL'].upper().find('CONTINUOUS ENROLLEE') > 0:
        return 'Continuous Enrollee'
    if row['LABEL'].upper().find('MOBILE') > 0:
        return 'Mobile'
    if row['LABEL'].upper().find('NON-AT RISK') > 0:
        return 'Non-At Risk'
    if row['LABEL'].upper().find('NON-CATE') > 0:
        return 'Non-CATE'
    if row['LABEL'].upper().find('NON-ELL') > 0 or row['LABEL'].upper().find('NON-EL') > 0:
        return 'Non-ELL'
    if row['LABEL'].upper().find('NON-ECON DISADV') > 0:
        return 'Non-Econ Disadv'
    if row['LABEL'].upper().find('NON-MIGRANT') > 0:
        return 'Non-Migrant'
    if row['LABEL'].upper().find('NON-SPECIAL ED') > 0:
        return 'Non-Special Ed'
    if row['LABEL'].upper().find('FIRST-YEAR-MONITORED-ELL') > 0 or row['LABEL'].upper().find('FIRST-YEAR-MONITORED-EL') > 0:
        return 'First-Year-Monitored-ELL'
    if row['LABEL'].upper().find('SECOND-YEAR-MONITORED-ELL') > 0 or row['LABEL'].upper().find('SECOND-YEAR-MONITORED-EL') > 0:
        return 'Second-Year-Monitored-ELL'
    if row['LABEL'].upper().find('AFRICAN AMERICAN') > 0:
        return 'African American'
    if row['LABEL'].upper().find('ALL STUDENTS') > 0:
        return 'All Students'
    if row['LABEL'].upper().find('AMERICAN INDIAN') > 0:
        return 'American Indian'
    if row['LABEL'].upper().find('ASIAN') > 0:
        return 'Asian'
    if row['LABEL'].upper().find('FEMALE') > 0:
        return 'Female'
    if row['LABEL'].upper().find('MALE') > 0:
        return 'Male'
    if row['LABEL'].upper().find('AT RISK') > 0:
        return 'At Risk'
    if row['LABEL'].upper().find('ECON DISADV') > 0:
        return 'Econ Disadv'
    if row['LABEL'].upper().find('HISPANIC') > 0:
        return 'Hispanic'
    if row['LABEL'].upper().find('PACIFIC ISLANDER') > 0:
        return 'Pacific Islander'
    if row['LABEL'].upper().find('SPECIAL ED') > 0:
        return 'Special Ed'
    if row['LABEL'].upper().find('WHITE') > 0:
        return 'White'
    if row['LABEL'].upper().find('TWO OR MORE RACES') > 0:
        return 'Two or more races'
    if row['LABEL'].upper().find('ELECTIVE-CATE') > 0:
        return 'Elective-CATE'
    if row['LABEL'].upper().find('CATE') > 0:
        return 'CATE'
    if row['LABEL'].upper().find('MIGRANT') > 0:
        return 'Migrant'
    if row['LABEL'].upper().find('ELL') > 0 or row['LABEL'].upper().find('EL,') > 0:
        return 'ELL'
    return row['LABEL']
        
df['group_state'] = df.apply(lambda x: parseGroup(x), axis = 1)
print(df['group_state'].drop_duplicates())

0                        All Students
18204                African American
32180                           White
48072                        Hispanic
66112                 American Indian
69472               Two or more races
80920                             ELL
97792                           Asian
106508               Pacific Islander
108116                         Female
126292                           Male
144456                    Econ Disadv
162564                     Special Ed
180300                        At Risk
228528            Continuous Enrollee
246568                         Mobile
13318011                      Migrant
13320019                  Non-Migrant
13338223                     Non-CATE
13356415              Non-Econ Disadv
13373991               Non-Special Ed
13392175                  Non-At Risk
13410263                      Non-ELL
13428463     First-Year-Monitored-ELL
13432063    Second-Year-Monitored-ELL
14179015                         CATE
16690413    

In [10]:
# parse subject
def parseSubject(row):
    if row['LABEL'].upper().find('MATH') > 0 or row['LABEL'].upper().find('ALGEBRA') > 0 or row['LABEL'].upper().find('GEOMETRY') > 0:
        return 'Math'
    if row['LABEL'].upper().find('READING') > 0 or row['LABEL'].upper().find('ELA') > 0 or row['LABEL'].upper().find('WRITING') > 0 or row['LABEL'].upper().find('ENGLISH') > 0:
        return 'ELA'
    if row['LABEL'].upper().find('SOCIAL STUDIES') > 0 or row['LABEL'].upper().find('HISTORY') > 0 or row['LABEL'].upper().find('GEOGRAPHY') > 0:
        return 'Social Studies'
    if row['LABEL'].upper().find('SCIENCE') > 0 or row['LABEL'].upper().find('BIOLOGY') > 0 or row['LABEL'].upper().find('PHYSICS') > 0 or row['LABEL'].upper().find('CHEMISTRY') > 0:
        return 'Science'
    return row['LABEL']

df['subject'] = df.apply(lambda x: parseSubject(x), axis = 1)
print(df['subject'].drop_duplicates())

0                     ELA
776896               Math
1817608           Science
8179486    Social Studies
Name: subject, dtype: object


In [11]:
# split dataset into numerator/denominator values
df_num = df[df.LABEL.str.contains('Numerator')]
print(df_num.shape)
df_denom = df[df.LABEL.str.contains('Denominator')]
print(df_denom.shape)

(15747651, 11)
(8089273, 11)


In [12]:
# parse performance level
def parsePerformance(row):
    if row['LABEL'].upper().find('APPROACHES GRADE LEVEL') > 0:
        return 'Approaches Grade Level'
    if row['LABEL'].upper().find('MASTERS GRADE LEVEL') > 0:
        return 'Masters Grade Level'
    if row['LABEL'].upper().find('MEETS GRADE LEVEL') > 0:
        return 'Meets Grade Level'
    return 'Phase-in 1 Level II or above'

df_num['performance_level'] = df_num.apply(lambda x: parsePerformance(x), axis = 1)
print(df_num['performance_level'].drop_duplicates())

4551              Approaches Grade Level
9102                   Meets Grade Level
13653                Masters Grade Level
53379646    Phase-in 1 Level II or above
Name: performance_level, dtype: object


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:
# drop unneeded columns
df_num = df_num[['CAMPUS', 'VALUE', 'year', 'grade', 'group_state', 'subject', 'performance_level']]
df_num = df_num.rename(columns = {'VALUE': 'num_at_level'})

def createProficientTF(row):
    if row['performance_level'] == 'Approaches Grade Level':
        return 0
    if row['performance_level'] == 'Phase-in 1 Level II or above' or row['performance_level'] == 'Masters Grade Level' or row['performance_level'] == 'Meets Grade Level':
        return 1
df_num['proficient_tf'] = df_num.apply(lambda x: createProficientTF(x), axis = 1)
print(df_num['proficient_tf'].drop_duplicates())

df_denom = df_denom[['CAMPUS', 'VALUE', 'year', 'grade', 'group_state', 'subject']]
df_denom = df_denom.rename(columns = {'VALUE': 'num_tested'})

4551    0
9102    1
Name: proficient_tf, dtype: int64


In [14]:
# roll up all results by subject/grade/group/year/campus/proficient_tf/performance_level
print(df_num.shape)
grouped_by = ['year', 'CAMPUS', 'subject', 'grade', 'group_state', 'proficient_tf', 'performance_level']
# change suppressed values to 0, so they aren't integers
df_num['num_at_level'] = [int(x) if int(x) >= 0 else 0 for x in df_num['num_at_level']]
df_num = df_num.groupby(grouped_by, as_index = False)['num_at_level'].sum()
print(df_num.shape)

(15747651, 8)
(13871505, 8)


In [15]:
print(df_denom.shape)
grouped_by.remove('proficient_tf')
grouped_by.remove('performance_level')
# change suppressed values to 0, so they aren't integers
df_denom['num_tested'] = [int(x) if int(x) >= 0 else 0 for x in df_denom['num_tested']]
df_denom = df_denom.groupby(grouped_by, as_index = False)['num_tested'].sum()
print(df_denom.shape)

(8089273, 6)
(7158273, 6)


In [16]:
# join num_at_level and num_tested values to each other
print(df_num.shape)
print(df_denom.shape)
df_joined = df_num.merge(df_denom, on = ['CAMPUS', 'year', 'subject', 'grade', 'group_state'], how='inner')
print(df_joined.head())
print(df_joined.shape)

(13871505, 8)
(7158273, 6)
   year   CAMPUS subject          grade       group_state  proficient_tf  \
0  2013  1902001     ELA  EOC Reading I  African American              1   
1  2013  1902001     ELA  EOC Reading I      All Students              1   
2  2013  1902001     ELA  EOC Reading I             Asian              1   
3  2013  1902001     ELA  EOC Reading I           At Risk              1   
4  2013  1902001     ELA  EOC Reading I              CATE              1   

              performance_level  num_at_level  num_tested  
0  Phase-in 1 Level II or above             0           0  
1  Phase-in 1 Level II or above            39          53  
2  Phase-in 1 Level II or above             0           0  
3  Phase-in 1 Level II or above            11          21  
4  Phase-in 1 Level II or above             7          10  
(13871505, 9)


In [17]:
# change suppressed values back to 0
def suppressNumAtLevel(row):
    if row['num_at_level'] == 0 and row['num_tested'] == 0:
        return 'N/A'
    return row['num_at_level']

def suppressNumTested(row):
    if row['num_at_level'] == 0 and row['num_tested'] == 0:
        return 'N/A'
    return row['num_tested']

df_joined['num_at_level'] = df_joined.apply(lambda x: suppressNumAtLevel(x), axis = 1)
df_joined['num_tested'] = df_joined.apply(lambda x: suppressNumTested(x), axis = 1)

In [18]:
# join campus data file
print(df_joined.shape)
df_school = df_school[['CAMPUS', 'CAMPNAME', 'DISTRICT', 'DISTNAME']]
df_joined = df_joined.merge(df_school, on = 'CAMPUS')
df_joined = df_joined.rename(columns = {
    'CAMPUS': 'school_id',
    'CAMPNAME': 'school',
    'DISTRICT': 'district_id',
    'DISTNAME': 'district'
})
print(df_joined.shape)
print(df_joined.head())

(13871505, 9)
(93924848, 12)
   year  school_id subject          grade       group_state  proficient_tf  \
0  2013    1902001     ELA  EOC Reading I  African American              1   
1  2013    1902001     ELA  EOC Reading I  African American              1   
2  2013    1902001     ELA  EOC Reading I  African American              1   
3  2013    1902001     ELA  EOC Reading I  African American              1   
4  2013    1902001     ELA  EOC Reading I  African American              1   

              performance_level num_at_level  num_tested      school  \
0  Phase-in 1 Level II or above          N/A           0  CAYUGA H S   
1  Phase-in 1 Level II or above          N/A           0  CAYUGA H S   
2  Phase-in 1 Level II or above          N/A           0  CAYUGA H S   
3  Phase-in 1 Level II or above          N/A           0  CAYUGA H S   
4  Phase-in 1 Level II or above          N/A           0  CAYUGA H S   

   district_id    district  
0         1902  CAYUGA ISD  
1         1

In [19]:
# drop all non-proficiency rows
print(df_joined.shape)
df_joined = df_joined[df_joined.proficient_tf == 1]
print(df.shape)

(93924848, 12)
(23836924, 11)


In [20]:
# convert num_at_level and num_tested to float
def toFloat(row, column):
    try:
        return float(row[column])
    except:
        return np.nan
df_joined['num_tested'] = df_joined.apply(lambda x: toFloat(x, 'num_tested'), axis = 1)
df_joined['num_at_level'] = df_joined.apply(lambda x: toFloat(x, 'num_at_level'), axis = 1)
print(df_joined.shape)

(71264898, 12)


In [21]:
# remove rows with no scores
print(df_joined.shape)
df_joined = df_joined[df_joined['num_at_level'] != 0]
df_joined = df_joined.dropna(subset=['num_tested', 'num_at_level'])
print(df_joined.shape)

(71264898, 12)
(45942206, 12)


In [22]:
# roll up into a single performance level
print(df_joined.shape)
grouped_by = ['year', 'district_id', 'district', 'school_id', 'school', 'subject', 'grade', 'group_state', 'proficient_tf']
df_joined = df_joined.groupby(grouped_by, as_index=False).agg({'num_tested': 'sum', 'num_at_level': 'sum'})
print(df_joined.shape)

(45942206, 12)
(5294671, 11)


In [23]:
# create 'All' grade rollup
grouped_by = ['year', 'district_id', 'district', 'school_id', 'school', 'subject', 'group_state', 'proficient_tf']
df_allgrades = df_joined.groupby(grouped_by, as_index=False)['num_at_level'].sum()
print(df_allgrades.shape)
df_allgrades2 = df_joined.groupby(grouped_by, as_index=False)['num_tested'].sum()
print(df_allgrades2.shape)
df_allgrades = df_allgrades.merge(df_allgrades2, on=grouped_by)
print(df_allgrades.shape)
del df_allgrades2
df_allgrades['grade'] = 'All Grades'

(2736294, 9)
(2736294, 9)
(2736294, 10)


In [24]:
# append 'All' grade rollup to df
print(df_joined.shape)
df_joined = df_joined.append(df_allgrades, ignore_index=True, sort=True)
print(df_joined.shape)
del df_allgrades

(5294671, 11)
(8030965, 11)


In [25]:
# create pct_at_level
df_joined['pct_at_level'] = df_joined['num_at_level'] / df_joined['num_tested']

In [26]:
print(df_joined.dtypes)
df_joined['district_id'] = df_joined['district_id'].astype(str)
df_joined['school_id'] = df_joined['school_id'].astype(str)
df_joined['proficient_tf'] = df_joined['proficient_tf'].astype(bool)
print(df_joined.dtypes)

district          object
district_id        int64
grade             object
group_state       object
num_at_level     float64
num_tested       float64
proficient_tf      int64
school            object
school_id          int64
subject           object
year              object
pct_at_level     float64
dtype: object
district          object
district_id       object
grade             object
group_state       object
num_at_level     float64
num_tested       float64
proficient_tf       bool
school            object
school_id         object
subject           object
year              object
pct_at_level     float64
dtype: object


In [28]:
print(df_joined['year'].drop_duplicates())

0          2013
698827     2014
1563004    2015
2132919    2016
3013723    2017
3589232    2018
4191776    2019
Name: year, dtype: object


In [27]:
# export data
df_joined.to_csv('./data/finalized/tx_proficiency.csv', index=False)